In [ ]:
from findspark import init
init()
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import * 
from utils import *


In [ ]:
spark = SparkSession.builder.appName("SparkTest").master("local[*]").getOrCreate()


In [ ]:
empSchema = StructType([ StructField("emp_no",LongType(),False),
                        StructField("birth_date",DateType(),False),
                        StructField("first_name",StringType(),True),
                        StructField("last_name",StringType(),True),
                        StructField("gender",StringType(),True),
                        StructField("hire_date",DateType(),False) ])

titlesSchema = StructType([ StructField("emp_no",LongType(),False),
                            StructField("title",StringType(),False),
                            StructField("from_date",DateType(),True),
                            StructField("to_date",DateType(),True) ])

In [ ]:
empsRDD = spark.sparkContext.parallelize(getInputData("data/employees.json"))
empsDF  = spark.read.schema(empSchema).json(empsRDD)
empsDF.createOrReplaceTempView("employees")

titlesRDD = spark.sparkContext.parallelize(getInputData("data/titles.json"))
titlesDF  = spark.read.schema(titlesSchema).json(titlesRDD)
titlesDF.createOrReplaceTempView("titles")

title = spark.sql("select * from titles") \
             .select('emp_no','title','from_date','to_date') \
             .where("from_date <= current_date and to_date > current_date")
title.cache()        
        
emp = spark.table("employees")
emp.cache()